Lista #3 MCMC
===

Thiago da Mota Souza - thiagosz@cos.ufrj.br

## Imports

In [227]:
import math
import numpy as np
import random
import time
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt

## Questão 1

Usando a mesma ideia da utilizada em sala de aula, podemos considerar o sorteio de pontos dentro de um quadrado de lado unitário que estão contidos em um triângulo $ABC$ de lados $\overline{AB}=1$, $\overline{AC}=1$ e $\angle{BAC} = \alpha = \pi/4$. A área desse triângulo é:

$$
\begin{equation}
Area(ABC) = \frac{\sin{(\alpha)}x1x1}{2} = \frac{\sqrt{2}}{4}
\end{equation}
$$

### Questão 1.1

Estabelecendo o vértice A como a origem do sistema de coordenadas, o eixo $x$ congruente ao lado $AC$ e o eixo $y$ perpendicular ao eixo $x$ como o de costume. Assim temos: $A = (0,0)$, $B=\sqrt{2}/2(1,1)$ e $C=(1,0)$

Podemos definir as funções determinísticas, $f_{AB}$ e $f_{BC}$, que descrevem os lados $AB$ e $BC$ respectivamente nos nos intervalos $x \in [0,1/2]$ e $x \in [1/2,1]$

$$
\begin{equation}
f_{AB} = \sqrt{2}x
\end{equation}
$$

$$
\begin{equation}
f_{BC} = -\sqrt{2}x + \sqrt{2}
\end{equation}
$$

E portanto vamos definir a função $g$ que descreve da sequinte forma

$$
g(x) = 
     \begin{cases}
       \sqrt{2}x & 0 \geq x < 1/2 \\
       -\sqrt{2}x + \sqrt{2}, & 1/2 < x \leq 1 \\
     \end{cases}
$$

> OBS: eu entendo que devido as constantes da função g, o que segue tem uma falha que é eu ter que usar o valor de $\sqrt{2}$ no computo de g. Não tenho tempo para bolar uma outra forma geométrica que escape à isso e vou seguir a questão ignorando esse fato que não altera o uso de Monte Carlo em si.

Utilizando a mesma ideia da usada para achar $\pi$ em sala de aula, podemos estimar a área em baixo dda função $g$ no intervalo $[0,1]$ Para isso devemos definir: $X,Y \sim Uniforme(0,1)$. Por tanto:

$$
\begin{equation}
\label{eq:Q-1-area-int}
Area(ABC) = \int_{x=0}^{1}g(x)dx = E[g(X)] = \frac{\sqrt(2)}{4}
\end{equation}
$$


### Questão 1.2

da questão anterior, temos que:

$$
\begin{equation}
\sigma_{g(X)}^2 = E[(g(X)- \mu_{g(x)})^2] = E[g(X)^2] - \mu_{g(X)}^2
\end{equation}
$$

A média, $\mu_{g(X)}$ foi construida a partir da área do triângulo e portanto já sabemos que seu valor é $\sqrt{2}{4}$, mas isso pode ser verificado por:

$$
\begin{multline}
\mu_{g(X)} = E[g(X)] = \int_{0}^{1/2}\sqrt{2}xdx + \int_{1/2}^{1}(-\sqrt{2}x + \sqrt{2})dx = \\
\frac{\sqrt{2}}{2}x^2\bigg\rvert_{0}^{1/2} + \sqrt(2)\left(-1/2x^2 + x\right)\bigg\rvert_{1/2}^{1} = \frac{\sqrt{2}}{8} \frac{\sqrt{2}}{8} = \frac{\sqrt{2}}{4}
\end{multline}
$$

Para o cálculo de $\sigma_{g(X)}^2$, temos:

$$
\begin{multline}
E[g(X)^2] = \int_{0}^{1/2}2x^2dx + \int_{1/2}^{1}(x^2 -4x + 2)dx = \\
\frac{2}{3}x^3\bigg\rvert_{0}^{1/2} + (1/3x^3 -2x^2 + 2x)\bigg\rvert_{1/2}^{1} = \frac{2}{24} + \frac{19}{24} = \frac{7}{8}
\end{multline}
$$

logo:
$$
\begin{equation}
\sigma_{g(X)}^2 = E[(g(X)- \mu_{g(x)})^2] = E[g(X)^2] - \mu_{g(X)}^2 = \frac{7}{8} - \frac{1}{8} = \frac{3}{4}
\end{equation}
$$


### Questão 1.3

da documentação da biblioteca [numpy](https://numpy.org/doc/stable/reference/random/bit_generators/mt19937.html) do python, temos que a função ```numpy.random.MT19937``` usa o algoritmo de Mersenne Twister.

In [205]:
def generate_sample(g_fun, sample_sz, a=0,b=1):
    """Gerador gnérico de amostras da função g aplicada sobre uma distribuição unbiforme X no intervalo [a,b]"""
    vectorized_g_fun = np.vectorize(g_fun)
    
    x_list = np.random.uniform(a,b,sample_sz)
    
    return vectorized_g_fun(x_list), x_list

In [216]:
def g_fun(x):
    if x < 0.5:
        return math.sqrt(2)*x
    return math.sqrt(2)*(-x+1)

### Questão 1.4

fazendo a correção para a definção de erro do meu estimador, uma vez que a V.A. que eu estou gerando não tem expectativa $\sqrt(2)$, mas sim $\sqrt(2)/4$ 

$$
\begin{equation}
\hat{e}_{n} = \frac{|M_n - \sqrt{2}/4|}{\sqrt{2}/4}
\end{equation}
$$

Essa é a função de erro MAE (Mean absolute error)

> Como sabemos que a média amostral tem uma variância, vou utilizar K-folds para visualizar a variância do estimador

In [217]:
def mae(y_true, y_pred):
    return abs(y_true - y_pred)/y_true
mae = np.vectorize(mae)

In [218]:
def sanple_mean_estimator(values):
    return np.mean(values)

In [219]:
def run_mc_estimator(sample_gen, sample_size, mae, estimator, true_val, num_folds=5):
    
    samples,_ = sample_gen(num_folds*sample_size)
    
    kf = KFold(n_splits=num_folds, shuffle=False)
    estimate_sample = []
    error_sample = []
    for fold,_ in kf.split(samples):
        data = samples[fold]
        estimate = estimator(data)
        error = mae(true_val, estimate)
    
        estimate_sample.append(estimate)
        error_sample.append(error)
        
    return estimate_sample, error_sample

In [220]:
def g_sample_generator(sample_sz):
    global g_fun
    return generate_sample(g_fun, sample_sz, 0,1)

def run_experiments(min_sample_sz, max_sample_sz):
    global g_sample_generator, run_mc_estimator, mae, sanple_mean_estimator
    
    true_val = math.sqrt(2)/4
    range_len = max_sample_sz - min_sample_sz + 1
    estimates = np.zeros([5, range_len])
    errors = np.zeros([5, range_len])
    
    for i,sample_sz in enumerate(range(min_sample_sz, max_sample_sz + 1)):
        estimates_for_n, error_samples_for_n = run_mc_estimator(g_sample_generator, sample_sz, mae, sanple_mean_estimator, true_val)
        estimates[:,i] = estimates_for_n
        errors[:,i] = error_samples_for_n
    return estimates, errors

In [ ]:
max_sample_size = int(1e6)
estimates, errors = run_experiments(1,max_sample_size)

In [ ]:
def plot_experiment1(estimates, error):

    true_val = true_val = math.sqrt(2)/4
    
    fig = plt.figure(figsize=(10, 8))
    plt.subplot(2,1,1)
    plt.plot(estimates.T, 'x', color='blue');
    plt.axhline(y=true_val, color='r', linestyle='-')
    plt.grid(True)
    plt.xlabel('tamanho da amostra')
    plt.ylabel('estimativa')
    
    plt.subplot(2,1,2)
    plt.plot(error.T, 'x', color='blue');
    plt.axhline(y=0, color='r', linestyle='-')
    plt.grid(True)
    plt.xlabel('tamanho da amostra')
    plt.ylabel('mae')
    
    fig.subplots_adjust(top=0.85)
    plt.show()

In [ ]:
plot_experiment1(estimates, errors)